In [127]:
from pymongo import MongoClient
import json
import pandas as pd
import numpy as np
import re


client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [25]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [26]:
# Vamos a hacer un listado con las empresas que no cumplen con nuestros requisitos para 
# tenerlas ubicadas y así poder establecer descartes en base a la cercanía a estas empresas.

companies_dislike = db.companies.find({'$and':
                                      [{'deadpooled_year':{'$eq':None}},{'founded_year':{'$lt':2008}},
                                       {"offices.latitude":{'$exists':True}},{"offices.longitude":
                                       {'$exists':True}},{"offices.latitude":{'$ne':None}},
                                       {"offices.longitude":{'$ne':None}}]})


In [27]:
# Convertimos el objeto pymongo en DataFrame

companies_dislike = pd.DataFrame(list(companies_dislike))
companies_dislike.head(2)

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8d,"{'term_code': None, 'price_currency_code': 'US...","[{'term_code': None, 'price_currency_code': 'U...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'permalink': 'reddit', 'name'...",Fri May 25 20:03:23 UTC 2007,http://www.crunchbase.com/company/digg,...,(415) 436-9638,"[{'permalink': 'digg', 'name': 'Digg'}]","[{'title': 'Public Relations', 'provider': {'p...","[{'title': 'CEO', 'person': {'first_name': 'An...","[{'available_sizes': [[[117, 150], 'assets/ima...","community, social, news, bookmark, digg, techn...",$45M,digg,Tue Nov 05 21:35:47 UTC 2013,[{'description': '<p>This video was originally...
1,52cdef7c4bab8bd675297d90,"{'term_code': 'cash', 'price_currency_code': '...",[],None,,,web,"[{'competitor': {'permalink': 'rescuetime', 'n...",Fri Jun 08 12:19:51 UTC 2007,http://www.crunchbase.com/company/postini,...,888.584.3150,"[{'permalink': 'postini-solutions', 'name': 'P...",[],"[{'title': 'Board', 'person': {'first_name': '...",[],,$0,None,Sat Aug 13 18:02:34 UTC 2011,[]


In [28]:
# Establecemos el index con el ID de mongodb

companies_dislike.set_index('_id', inplace=True)
companies_dislike.head(2)

,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,deadpooled_day,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
_id,,,,,,,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297d8d,"{'term_code': None, 'price_currency_code': 'US...","[{'term_code': None, 'price_currency_code': 'U...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'permalink': 'reddit', 'name'...",Fri May 25 20:03:23 UTC 2007,http://www.crunchbase.com/company/digg,None,...,(415) 436-9638,"[{'permalink': 'digg', 'name': 'Digg'}]","[{'title': 'Public Relations', 'provider': {'p...","[{'title': 'CEO', 'person': {'first_name': 'An...","[{'available_sizes': [[[117, 150], 'assets/ima...","community, social, news, bookmark, digg, techn...",$45M,digg,Tue Nov 05 21:35:47 UTC 2013,[{'description': '<p>This video was originally...
52cdef7c4bab8bd675297d90,"{'term_code': 'cash', 'price_currency_code': '...",[],None,,,web,"[{'competitor': {'permalink': 'rescuetime', 'n...",Fri Jun 08 12:19:51 UTC 2007,http://www.crunchbase.com/company/postini,None,...,888.584.3150,"[{'permalink': 'postini-solutions', 'name': 'P...",[],"[{'title': 'Board', 'person': {'first_name': '...",[],,$0,None,Sat Aug 13 18:02:34 UTC 2011,[]


In [29]:
#Eliminamos las columnas que no nos son de utilidad

companies_dislike.drop(companies_dislike.columns[[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 20, 21, 22, 24, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 40
]], axis=1, inplace=True)

companies_dislike.head(2)

,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised
_id,,,,,,,,,,,
52cdef7c4bab8bd675297d8d,news,None,user driven social content website,2004,"[{'round_code': 'b', 'funded_day': 1, 'id': 1,...",None,Digg,60.0,"[{'latitude': 37.764726, 'zip_code': '94107', ...","community, social, news, bookmark, digg, techn...",$45M
52cdef7c4bab8bd675297d90,web,None,None,1999,[],None,Postini,NaN,"[{'latitude': 37.506885, 'zip_code': '94070', ...",,$0


In [30]:
# Ahora vamos a establecer la geolocalizacion, para luego ponerla en el geoindex

def getFirst(data):
    data = data['offices']

    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "totalOffices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "oficina_principal": principal
    }


comp_dislike_off = companies_dislike[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [37]:
# Incluimos en el DataFrame esta información

companies_dislike = pd.concat([companies_dislike, comp_dislike_off], axis=1)
companies_dislike.head(2)

,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised,lat,lng,oficina_principal,totalOffices,lat,lng,oficina_principal,totalOffices
_id,,,,,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297d8d,news,None,user driven social content website,2004,"[{'round_code': 'b', 'funded_day': 1, 'id': 1,...",None,Digg,60.0,"[{'latitude': 37.764726, 'zip_code': '94107', ...","community, social, news, bookmark, digg, techn...",$45M,37.764726,-122.394523,"{'coordinates': [-122.394523, 37.764726], 'typ...",1,37.764726,-122.394523,"{'coordinates': [-122.394523, 37.764726], 'typ...",1
52cdef7c4bab8bd675297d90,web,None,None,1999,[],None,Postini,NaN,"[{'latitude': 37.506885, 'zip_code': '94070', ...",,$0,37.506885,-122.247573,"{'coordinates': [-122.247573, 37.506885], 'typ...",1,37.506885,-122.247573,"{'coordinates': [-122.247573, 37.506885], 'typ...",1


In [42]:
# Esta lista la ultilizaré en el siguiente paso, ya que no veo la forma de establecer áreas negras para realizar
# descartes, en su lugar, al obtener la lista de empresas cercanas, voy a descartar esa empresa si una de estas
# empresas aparece en el listado

names_companies_dislike = list(companies_dislike['name'])

In [32]:
# Ahora guardamos el dataframe en un json

companies_dislike.to_json('./companies_dislike.json', orient='records', default_handler=str)

In [107]:
# Hacemos un filtro de las empresas que pueden encajar con nuestros requisitos


filtered_companies = db.companies.find({'$and': 
                                        [{'deadpooled_year':{'$eq':None}}, 
                                         {'founded_year':{'$gte':2008}},
                                         {'total_money_raised': {'$ne': "$0"}},
                                         {'total_money_raised': {'$ne': None}},
                                         {"offices.latitude":{'$exists':True}},
                                         {"offices.longitude":{'$exists':True}},
                                         {"offices.latitude":{'$ne':None}},
                                         {"offices.longitude":{'$ne':None}},
                                         {'category_code':{'$not':{'$eq':None}}}, 
                                         {'category_code':{'$not':{'$eq':'education'}}},
                                         {'category_code':{'$not':{'$eq':'fashion'}}}, 
                                         {'category_code':{'$not':{'$eq':'health'}}},
                                         {'category_code':{'$not':{'$eq':'hospitality'}}}, 
                                         {'category_code':{'$not':{'$eq':'legal'}}},
                                         {'category_code':{'$not':{'$eq':'medical'}}}, 
                                         {'category_code':{'$not':{'$eq':'other'}}},
                                         {'category_code':{'$not':{'$eq':'real_estate'}}}, 
                                         {'category_code':{'$not':{'$eq':'travel'}}}
                                        ]})


In [108]:
# Convertimos el objeto pymongo en DataFrame

filtered_companies = pd.DataFrame(list(filtered_companies))
print(filtered_companies.shape)
filtered_companies.head(2)


(319, 42)


,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297e30,None,[],None,,http://goingon.com/blog,software,"[{'competitor': {'permalink': 'ning', 'name': ...",Fri Jul 20 04:55:26 UTC 2007,http://www.crunchbase.com/company/goingon,...,415-200-4606,"[{'permalink': 'goingon', 'name': 'GoingOn'}, ...",[],"[{'title': 'CEO', 'person': {'first_name': 'Jo...",[],"socialnetworking, enterprise, social-media, so...",$8.5M,GoingOn_Network,Thu Apr 04 09:03:17 UTC 2013,[]
1,52cdef7c4bab8bd675297ee1,None,[],,,http://blog.cloudant.com/,enterprise,[],Fri Aug 17 12:40:06 UTC 2007,http://www.crunchbase.com/company/cloudant,...,(857) 400-9900,[],[],"[{'title': 'Founder & President', 'person': {'...",[],"cloud-computing, big-data, databases, software...",$18.3M,cloudant,Mon Nov 11 19:25:28 UTC 2013,[{'description': '<p>Why Cloudant?: Mike Mille...


In [109]:
# Establecemos el index con el ID de mongodb

filtered_companies.set_index('_id', inplace=True)
filtered_companies.head(2)

,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,deadpooled_day,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
_id,,,,,,,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297e30,None,[],None,,http://goingon.com/blog,software,"[{'competitor': {'permalink': 'ning', 'name': ...",Fri Jul 20 04:55:26 UTC 2007,http://www.crunchbase.com/company/goingon,None,...,415-200-4606,"[{'permalink': 'goingon', 'name': 'GoingOn'}, ...",[],"[{'title': 'CEO', 'person': {'first_name': 'Jo...",[],"socialnetworking, enterprise, social-media, so...",$8.5M,GoingOn_Network,Thu Apr 04 09:03:17 UTC 2013,[]
52cdef7c4bab8bd675297ee1,None,[],,,http://blog.cloudant.com/,enterprise,[],Fri Aug 17 12:40:06 UTC 2007,http://www.crunchbase.com/company/cloudant,None,...,(857) 400-9900,[],[],"[{'title': 'Founder & President', 'person': {'...",[],"cloud-computing, big-data, databases, software...",$18.3M,cloudant,Mon Nov 11 19:25:28 UTC 2013,[{'description': '<p>Why Cloudant?: Mike Mille...


In [110]:
#Eliminamos las columnas que no nos son de utilidad

filtered_companies.drop(filtered_companies.columns[[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 20, 21, 22, 24, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 40
]], axis=1, inplace=True)

filtered_companies.head(2)

,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised
_id,,,,,,,,,,,
52cdef7c4bab8bd675297e30,software,None,Private Academic Networks,2008,"[{'round_code': 'seed', 'funded_day': 1, 'id':...",None,GoingOn,40.0,"[{'latitude': 37.782263, 'zip_code': '94111', ...","socialnetworking, enterprise, social-media, so...",$8.5M
52cdef7c4bab8bd675297ee1,enterprise,None,NoSQL database managed service,2008,"[{'round_code': 'a', 'funded_day': None, 'id':...",None,Cloudant,70.0,"[{'latitude': 42.35892, 'zip_code': '02111', '...","cloud-computing, big-data, databases, software...",$18.3M


In [111]:
# Ahora vamos a establecer la geolocalizacion, para luego ponerla en el geoindex

fil_comp_off = filtered_companies[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [112]:
# Incluimos en el DataFrame esta información

filtered_companies = pd.concat([filtered_companies, fil_comp_off], axis=1)
filtered_companies.head(2)

,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised,lat,lng,oficina_principal,totalOffices
_id,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297e30,software,None,Private Academic Networks,2008,"[{'round_code': 'seed', 'funded_day': 1, 'id':...",None,GoingOn,40.0,"[{'latitude': 37.782263, 'zip_code': '94111', ...","socialnetworking, enterprise, social-media, so...",$8.5M,37.782263,-122.392142,"{'coordinates': [-122.392142, 37.782263], 'typ...",1
52cdef7c4bab8bd675297ee1,enterprise,None,NoSQL database managed service,2008,"[{'round_code': 'a', 'funded_day': None, 'id':...",None,Cloudant,70.0,"[{'latitude': 42.35892, 'zip_code': '02111', '...","cloud-computing, big-data, databases, software...",$18.3M,42.358920,-71.057810,"{'coordinates': [-71.05781, 42.35892], 'type':...",1


In [113]:
# Ahora vamos a normalizar la columna 'total_money_raised'

In [126]:
lista = list(filtered_companies['total_money_raised'])
# tipos moneda --> $, £, €, C$ (empiezan por este valor y terminan por k, M ó B) - C$ = dolar canadiense 
# Exchange rates from https://www.federalreserve.gov/releases/g5a/current/
# 2018 values --> C$ = 1.2957 - € = 1.1817 - £ = 1.3363 

lista

['$8.5M',
 '$18.3M',
 '$4.8M',
 '$1.83M',
 '$7.1M',
 '$8M',
 '$16.8M',
 '$7M',
 '$58.8M',
 '$5M',
 '$17.1M',
 '$100k',
 '€19M',
 '$1M',
 '$9M',
 '$11M',
 '$35M',
 '$12M',
 '$4M',
 '$8.3M',
 '$6.64M',
 '$800k',
 '$30.5M',
 '$22.5M',
 '$250k',
 '€300k',
 '$1.25M',
 '$100M',
 '$1.3M',
 '$110k',
 '$4M',
 '$25M',
 '$8.75M',
 '$2.35M',
 '$175M',
 '£3M',
 '$8.1M',
 '$200k',
 '$21.8M',
 '$2.71M',
 '$18.7M',
 '$500k',
 '$28.5M',
 'C$12M',
 '$3.8M',
 '€250k',
 '$560k',
 '€500k',
 '$1.5M',
 '$2.8M',
 '$10k',
 '$55k',
 '$1M',
 '$350k',
 'C$350k',
 '$10.1M',
 '$3.8M',
 '$1.1M',
 '$3.8M',
 '$28.4M',
 '$31.4M',
 '$7.48M',
 '$25k',
 '$250k',
 '$125k',
 '$24M',
 '€3.04M',
 '$340k',
 '$44.8M',
 '£500k',
 '$8.09M',
 '$43M',
 '$930k',
 '$20k',
 '$250k',
 '$8.02M',
 '$3.1M',
 '$1.31M',
 '$800k',
 '$1M',
 '$3.5M',
 'C$500k',
 '€70k',
 '$10.6M',
 '$4M',
 '$4.2M',
 '$200k',
 '$1M',
 '$17M',
 '$43M',
 '$8.09M',
 '$930k',
 '$20k',
 '$250k',
 '$8.02M',
 '$3.1M',
 '$1.31M',
 '$800k',
 '$1M',
 '$3.5M',
 'C$500k',


In [254]:
def exchangeMoney (value):
    ex_rate = {'C$':1.2957, '€':1.1817, '£':1.3363}
    amount = {'k':1000, 'M':1000000, 'B': 1000000000}
    value_finder = float(re.findall('[0-9]*\.[0-9]|[0-9]\d*',value)[0])
    abb = ('k','K')
    
    if value.endswith(abb) & value.startswith("$"):
        r_amt = (value_finder * (amount['k']))    
    elif value.endswith(abb) & value.startswith("C$"):
        r_amt = (value_finder * (amount['k']) * (ex_rate['C$']))    
    elif value.endswith(abb) & value.startswith("€"):
        r_amt = (value_finder * (amount['k']) * (ex_rate['€']))      
    elif value.endswith(abb) & value.startswith("£"):
        r_amt = (value_finder * (amount['k']) * (ex_rate['£']))
                
    elif value.endswith('M') & value.startswith("$"):
        r_amt = (value_finder * (amount['M']))             
    elif value.endswith('M') & value.startswith("C$"):
        r_amt = (value_finder * (amount['M']) * (ex_rate['C$']))    
    elif value.endswith('M') & value.startswith("€"):
        r_amt = (value_finder * (amount['M']) * (ex_rate['€']))    
    elif value.endswith('M') & value.startswith("£"):
        r_amt = (value_finder * (amount['M']) * (ex_rate['£'])) 
    
    elif value.endswith('B') & value.startswith("$"):
        r_amt = (value_finder*(amount['B']))
    elif value.endswith('B') & value.startswith("C$"):
        r_amt = (value_finder*(amount['B'])*(ex_rate['C$']))   
    elif value.endswith('B') & value.startswith("€"):
        r_amt = (value_finder*(amount['B'])*(ex_rate['€']))    
    elif value.endswith('B') & value.startswith("£"):
        r_amt = (value_finder*(amount['B'])*(ex_rate['£'])) 

    return r_amt
    
    
    


In [258]:
toFind = filtered_companies['total_money_raised'][35]
test = exchangeMoney(toFind)
print("valor: ", toFind)
print("Prueba: ", test)



valor:  £3M
Prueba:  4008900.0


In [ ]:
"""
Códigos de terminal para importar a mongodb las bases de datos
---------------------------------------------------------------


mongoimport --db companies --collection companies_dislike --file companies_dislike.json --jsonArray

mongoimport --db companies --collection filtered_companies --file filtered_companies.json --jsonArray

"""

In [60]:
filtered_companies.to_json('./filtered_companies.json', orient='records', default_handler=str)